In [3]:
"""Import api key """

import getpass
import os
import cv2
import base64
import json
import time
import numpy as np
from openai import OpenAI
import re
import textwrap



In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()
client = OpenAI()

In [7]:
# Path to the directory containing session folders
base_directory = "./data/demo_data"

# Application name
app_name = "App1"

# Function to encode an image to a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Function to display base64-encoded images using OpenCV
def show_base64_image(base64_string):
    image_data = base64.b64decode(base64_string)
    np_array = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Collect results for documentation
results = {
    "app_name": app_name,
    "prompt": """Context:
You are given a lot of screenshots related to a user's interaction with our application.
Your task is to analyze this data and provide a detailed description of the user's steps, actions, and the sequence in which they occurred.
The goal is to understand the user's behavior and identify any issues or patterns.

Instructions:

1. Detailed Steps:
    Based on your analysis, provide a step-by-step description of the user's journey within the application. For each step, include the following details:

    a. Action: What the user did (e.g., clicked a button, entered text).
    b. Context: Where in the application the action took place (e.g., on which screen or page).
    c. Outcome: What happened as a result of the action (e.g., navigated to a new screen, received an error message).
    d. Corroboration: Reference the screenshots that support this step.

2. Issues and Observations:
    Highlight any potential issues or unusual behaviors you observe. This could include errors, unexpected actions, or anything that deviates from typical usage patterns.

3. Summary:
    Conclude with a summary of the user's overall session, including the main goals they appeared to be pursuing and any challenges they encountered.
""",
    "sessions": []  # This will hold the results of each session
}

# Loop through each session directory
for session_id in os.listdir(base_directory):
    print(session_id)
    session_path = os.path.join(base_directory, session_id, session_id)
    if os.path.isdir(session_path):
        # Collect and encode all images in the session folder
        image_files = sorted(
            [
                f
                for f in os.listdir(session_path)
                if f.endswith(".jpeg") or f.endswith(".png")
            ],
            key=lambda x: int(x.split('.')[0]),
            # key=lambda x: int(re.search(r'sr_(\d+)', x).group(1))  # Sort numerically by timestamp
        )
        print(image_files)
        base64_images = [encode_image(os.path.join(session_path, img)) for img in image_files]

        # Create the prompt with the encoded images
        PROMPT_MESSAGES = [
            {
                "role": "user",
                "content": [
                    results["prompt"],
                    
                    *map(lambda x: {"image": x, "resize": 768}, base64_images),
                ],
            },
        ]

        # OpenAI API parameters
        params = {
            "model": "gpt-4o",
            "messages": PROMPT_MESSAGES,
            "max_tokens": 4000,
        }

        # Send the request and record the response time
        start_time = time.time()
        try:
            result = client.chat.completions.create(**params)
            end_time = time.time()

            # Extract response details
            response_content = result.choices[0].message.content
            tokens_used = result.usage.total_tokens if hasattr(result.usage, 'total_tokens') else "N/A"
            formatted_response = textwrap.indent(response_content, "    ")
            # Log the session data
            session_result = {
                "session_id": session_id,
                "response": formatted_response,
                "tokens_consumed": tokens_used,
                "response_time_seconds": round(end_time - start_time, 2)
            }

            # Append the session result to the higher-level results
            results["sessions"].append(session_result)
            print(f"Processed session: {session_id}")

        except Exception as e:
            print(f"Error processing session {session_id}: {e}")

# Save the results to a JSON file
output_file = f"session_analysis_results_{app_name}.json"
with open(output_file, "w") as json_file:
    json.dump(results, json_file, indent=4)

print(f"Results saved to {output_file}")

137412742
['2024101708012127365457.jpeg', '2024101708012182492511.jpeg', '2024101708015127757782.jpeg']
Processed session: 137412742
2244972274
['202410170907170839136.jpeg', '2024101709071744433879.jpeg', '2024101709072117896740.jpeg', '2024101709072699931895.jpeg', '2024101709073626887074.jpeg', '2024101709073697844048.jpeg', '2024101709074354853269.jpeg', '2024101709074706847918.jpeg', '2024101709085189430358.jpeg', '2024101709085316417139.jpeg']
Processed session: 2244972274
79394014
['2024101705434594753508.jpeg', '2024101705434896620372.jpeg', '2024101705434957815975.jpeg', '2024101705440147228555.jpeg', '2024101705440627065457.jpeg', '2024101705440739492511.jpeg']
Processed session: 79394014
2046726162
['2024101709053437153508.jpeg', '2024101709053759820372.jpeg', '2024101709053826128555.jpeg', '2024101709054555465457.jpeg']
Processed session: 2046726162
1884322672
['2024101708014343063848.jpeg', '2024101708014413523384.jpeg', '2024101708014676575066.jpeg', '20241017080147512985

In [15]:
for result in results["sessions"]:
    result["response"] = textwrap.indent(response_content, "    ")
    
output_file = f"session_analysis_results_{app_name}12.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, indent=4, ensure_ascii=False)


In [3]:
"""Simple data"""
import os
import cv2
import numpy as np
logs_path = 'logsRegUnziped.txt'
logs=''
with open(logs_path, 'r') as file:
    # Read the content of the file
    content = file.read()
    logs = content

base64_image =[]
def show_base64_image(base64_string):
    image_data = base64.b64decode(base64_string)
    # Convert the decoded image into a numpy array
    np_array = np.frombuffer(image_data, np.uint8)
    # Decode the image from the numpy array
    image = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
    # Display the image using OpenCV
    cv2.imshow('Image', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
# Define the path to the folder containing images
folder_path = "./data/linkedinSessoinSDK"
folder_path2 = "./data/slackSessionSDK"
# Get all JPEG or PNG files in the folder and sort them
image_files = sorted(
    [f for f in os.listdir(folder_path2) if f.endswith(".jpeg") or f.endswith(".png")],
    key=lambda x: int(x.split('.')[0])  # Extract numeric part for sorting

)

# Loop through the sorted file list
for filename in image_files:
    img_path = os.path.join(folder_path2, filename)
    base64_str= encode_image(img_path)
    base64_image.append(base64_str)  
    print(filename)

    show_base64_image(base64_str)




1.jpeg
2.jpeg
3.jpeg
4.jpeg
5.jpeg
6.jpeg


In [ ]:

PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            """"
Context:
    You are given a lot of screenshots related to a user's interaction with our application. Your task is to analyze this data and provide a detailed description of the user's steps, actions, and the sequence in which they occurred. The goal is to understand the user's behavior and identify any issues or patterns.

Instructions:

1. Detailed Steps:
    Based on your analysis, provide a step-by-step description of the user's journey within the application. For each step, include the following details:

    a. Action: What the user did (e.g., clicked a button, entered text).
    b. Context: Where in the application the action took place (e.g., on which screen or page).
    c. Outcome: What happened as a result of the action (e.g., navigated to a new screen, received an error message).
    d. Corroboration: Reference the screenshots that support this step.

2. Issues and Observations:
    Highlight any potential issues or unusual behaviors you observe. This could include errors, unexpected actions, or anything that deviates from typical usage patterns.

3. Summary:
    Conclude with a summary of the user's overall session, including the main goals they appeared to be pursuing and any challenges they encountered.
""",
    *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
params = {
    "model": "gpt-4o",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 4000,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

In [ ]:
'''The real test with both the attachments and logs'''
prompt = f"""
Context:
    You are given session logs and screenshots related to a user's interaction with our application. Your task is to analyze this data and provide a detailed description of the user's steps, actions, and the sequence in which they occurred. The goal is to understand the user's behavior and identify any issues or patterns.

Instructions:

1. Session Logs:
    Review the session logs provided. These logs contain timestamps, user actions, and other relevant events. Identify key actions performed by the user and the sequence of these actions.
    {logs}
2. Screenshots:
    Examine the attached screenshots. These images correspond to specific points in the user's session and should be used to corroborate the actions described in the logs.

3. Detailed Steps:
    Based on your analysis, provide a step-by-step description of the user's journey within the application. For each step, include the following details:

    a. Action: What the user did (e.g., clicked a button, entered text).
    b. Context: Where in the application the action took place (e.g., on which screen or page).
    c. Outcome: What happened as a result of the action (e.g., navigated to a new screen, received an error message).
    d. Corroboration: Reference the log entries and screenshots that support this step.

4. Issues and Observations:
    Highlight any potential issues or unusual behaviors you observe. This could include errors, unexpected actions, or anything that deviates from typical usage patterns.

5. Summary:
    Conclude with a summary of the user's overall session, including the main goals they appeared to be pursuing and any challenges they encountered.
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text", 
         "text": prompt
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[0]}",
          },
        },
        {
          "type": "image_url",
          "image_url": { 
            "url":f"data:image/jpeg;base64,{base64_image[1]}",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[2]}",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[3]}",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[4]}",
          },
        },
          {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[5]}",
          },
        },
          {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[7]}",
          },
        },
      ],
    }
  ],
  max_tokens=4000,
)

print(response)

In [ ]:
'''The real test with logs only'''
logs_prompt = f"""
Context:
    You are given session logs  related to a user's interaction with our application. Your task is to analyze this data and provide a detailed description of the user's steps, actions, and the sequence in which they occurred. The goal is to understand the user's behavior and identify any issues or patterns.

Instructions:

1. Session Logs:
    Review the session logs provided. These logs contain timestamps, user actions, and other relevant events. Identify key actions performed by the user and the sequence of these actions.
    {logs}

2. Detailed Steps:
    Based on your analysis, provide a step-by-step description of the user's journey within the application. For each step, include the following details:

    a. Action: What the user did (e.g., clicked a button, entered text).
    b. Context: Where in the application the action took place (e.g., on which screen or page).
    c. Outcome: What happened as a result of the action (e.g., navigated to a new screen, received an error message).
    d. Corroboration: Reference the log entries and screenshots that support this step.

3. Issues and Observations:
    Highlight any potential issues or unusual behaviors you observe. This could include errors, unexpected actions, or anything that deviates from typical usage patterns.

4. Summary:
    Conclude with a summary of the user's overall session, including the main goals they appeared to be pursuing and any challenges they encountered.
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text", 
         "text": logs_prompt
        }
      ],
    }
  ],
  max_tokens=4000,
)

print(response)

In [8]:
'''The real test with the attachments only'''
prompt = f"""
Context:
    You are given session  screenshots related to a user's interaction with our application. Your task is to analyze this data and provide a detailed description of the user's steps, actions, and the sequence in which they occurred. The goal is to understand the user's behavior and identify any issues or patterns.

Instructions:

1. Screenshots:
    Examine the attached screenshots. These images correspond to specific points in the user's session.

2. Detailed Steps:
    Based on your analysis, provide a step-by-step description of the user's journey within the application. For each step, include the following details:

    a. Action: What the user did (e.g., clicked a button, entered text).
    b. Context: Where in the application the action took place (e.g., on which screen or page).
    c. Outcome: What happened as a result of the action (e.g., navigated to a new screen, received an error message).
    d. Corroboration: Reference the screenshots that support this step.

3. Issues and Observations:
    Highlight any potential issues or unusual behaviors you observe. This could include errors, unexpected actions, or anything that deviates from typical usage patterns.

4. Summary:
    Conclude with a summary of the user's overall session, including the main goals they appeared to be pursuing and any challenges they encountered.
"""

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text", 
          "text": prompt
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[0]}",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[1]}",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[2]}",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[3]}",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[4]}",
          },
        },
          {
          "type": "image_url",
          "image_url": {
            "url":f"data:image/jpeg;base64,{base64_image[5]}",
          },
        }
      ],
    }
  ],
  max_tokens=4000,
)

print(response)

ChatCompletion(id='chatcmpl-A5Jq4N1fiSNGpz7jrlA0ESkxfpQpL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The screenshots show a workspace in the Slack application, an instant messaging platform often used by teams and organizations for communication and collaboration. Here\'s a summary of the contents from the screenshots:\n\n1. **Image 1: Home Tab**  \n   - Displays the name "Instabug" and shows the main interface.\n   - Contains starred channels and other channels related to different topics like `be_apm`, `be_monitoring`, `be_newrelic_sync`, etc.\n   - At the top, there are options for "Catch up," "Threads," "Later," and "Drafts & sent items."\n\n2. **Image 2: `squad-ai` Channel**  \n   - Shows a conversation with messages from users including Karim, Kirollos Samy, and Salma Yasser.\n   - Topics discussed:\n     - Receiving code-diff and natural language fix.\n     - AutoFix evaluation and IOS dataset thread.\n     - Review of 

In [9]:
print(response.choices[0].message.content)

The screenshots show a workspace in the Slack application, an instant messaging platform often used by teams and organizations for communication and collaboration. Here's a summary of the contents from the screenshots:

1. **Image 1: Home Tab**  
   - Displays the name "Instabug" and shows the main interface.
   - Contains starred channels and other channels related to different topics like `be_apm`, `be_monitoring`, `be_newrelic_sync`, etc.
   - At the top, there are options for "Catch up," "Threads," "Later," and "Drafts & sent items."

2. **Image 2: `squad-ai` Channel**  
   - Shows a conversation with messages from users including Karim, Kirollos Samy, and Salma Yasser.
   - Topics discussed:
     - Receiving code-diff and natural language fix.
     - AutoFix evaluation and IOS dataset thread.
     - Review of specs for initiating fixes.
     - Documentation of evaluation processes and feature implementation.

3. **Image 3: `squad-ai` Thread**  
   - Another look into a `squad-ai` 

In [13]:
from IPython.display import display, Image

import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import time


In [19]:
video = cv2.VideoCapture("data/slack session.mp4")

base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64Frames), "frames read.")

994 frames read.


In [1]:
display_handle = display(None, display_id=True)
for img in base64Frames[0::50]:
    display_handle.update(Image(data=base64.b64decode(img.encode("utf-8"))))
    time.sleep(0.25)

None

NameError: name 'base64Frames' is not defined

In [23]:
'''Test with the video frames'''
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            """"
Context:
    You are given a lot of frames related to a user's interaction with our application. Your task is to analyze this data and provide a detailed description of the user's steps, actions, and the sequence in which they occurred. The goal is to understand the user's behavior and identify any issues or patterns.

Instructions:

1. Detailed Steps:
    Based on your analysis, provide a step-by-step description of the user's journey within the application. For each step, include the following details:

    a. Action: What the user did (e.g., clicked a button, entered text).
    b. Context: Where in the application the action took place (e.g., on which screen or page).
    c. Outcome: What happened as a result of the action (e.g., navigated to a new screen, received an error message).
    d. Corroboration: Reference the screenshots that support this step.

2. Issues and Observations:
    Highlight any potential issues or unusual behaviors you observe. This could include errors, unexpected actions, or anything that deviates from typical usage patterns.

3. Summary:
    Conclude with a summary of the user's overall session, including the main goals they appeared to be pursuing and any challenges they encountered.
""",
    *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
params = {
    "model": "gpt-4o",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 4000,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

### Detailed Steps:

#### Step 1:
**Action:** The user opened the Instabug application.
**Context:** Home screen of the application.
**Outcome:** Displayed the list of channels on the screen.
**Corroboration:** Screenshot 1.

#### Step 2:
**Action:** The user selected the "be_apm" channel.
**Context:** Channels list on the home screen.
**Outcome:** Navigated to the "be_apm" channel view.
**Corroboration:** Screenshot 2.

#### Step 3:
**Action:** The "be_apm" channel began to load its content.
**Context:** "be_apm" channel screen.
**Outcome:** Initially displayed an empty loading screen.
**Corroboration:** Screenshot 3.

#### Step 4:
**Action:** The user waited for the content to load.
**Context:** "be_apm" channel screen.
**Outcome:** The channel's recent messages are displayed.
**Corroboration:** Screenshot 4.

#### Step 5:
**Action:** The user browsed through the "be_apm" channel messages.
**Context:** "be_apm" channel screen.
**Outcome:** Scrolled to view more messages.
**Corroborat

In [24]:
print(result)

ChatCompletion(id='chatcmpl-A5K1BnO9h4LkgBpq6Kf1n07NKwJwc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='### Detailed Steps:\n\n#### Step 1:\n**Action:** The user opened the Instabug application.\n**Context:** Home screen of the application.\n**Outcome:** Displayed the list of channels on the screen.\n**Corroboration:** Screenshot 1.\n\n#### Step 2:\n**Action:** The user selected the "be_apm" channel.\n**Context:** Channels list on the home screen.\n**Outcome:** Navigated to the "be_apm" channel view.\n**Corroboration:** Screenshot 2.\n\n#### Step 3:\n**Action:** The "be_apm" channel began to load its content.\n**Context:** "be_apm" channel screen.\n**Outcome:** Initially displayed an empty loading screen.\n**Corroboration:** Screenshot 3.\n\n#### Step 4:\n**Action:** The user waited for the content to load.\n**Context:** "be_apm" channel screen.\n**Outcome:** The channel\'s recent messages are displayed.\n**Corroboration:** Scree